In [83]:
import pandas as pd
import re

data = "dbpedia_14"
text_len = 60
model = "transformeral"
layer = 5
epoch = 10
lr = 0.0001
fix_previous_layer = True
#for mask in range(1,1+layer):
mask = 5
save_path = f"ckpt/{data}_{model}_l{layer}_pad{text_len}_m{mask}/"  
#load_path = f"ckpt/{data}_{model}_l{layer}_pad{text_len}_m{mask-1}/" 
out_path = f"result/0117/fix train adapt/{data}_{model}_l{layer}ad_pad{text_len}_m{mask}/"
log = f"{out_path}/{data}_{model}_l{layer}.log"

with open(log,mode='r') as log:
    buffer = log.readlines()
    df = pd.DataFrame(buffer,columns=["log"])
    df

In [85]:
class history(object):
    def __init__(self):
        self.auc = {}
        self.acc = {}
        self.entr = {}
        for threshold in [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]:
            self.auc[threshold] = []
            self.acc[threshold] = []
            self.entr[threshold] = []
            
result = history()
for line in buffer:
    match = re.match('Test Epoch(.)*', line)
    if match!=None:
        print(match.group())
        match = re.findall("\d?\.\d+",match.group(0))
        print(match)
        threshold = float(match[0])
        result.acc[threshold].append(match[1])
        result.auc[threshold].append(match[2])
        result.entr[threshold].append(match[3])

Test Epoch0 threshold 0.1 Acc 0.9756714285714285, AUC 0.9981935620307922, avg_entr 0.004088811110705137
['0.1', '0.9756714285714285', '0.9981935620307922', '0.004088811110705137']
Test Epoch0 threshold 0.2 Acc 0.9747428571428571, AUC 0.9983512759208679, avg_entr 0.008886891417205334
['0.2', '0.9747428571428571', '0.9983512759208679', '0.008886891417205334']
Test Epoch0 threshold 0.3 Acc 0.9745857142857143, AUC 0.9983507394790649, avg_entr 0.009163127280771732
['0.3', '0.9745857142857143', '0.9983507394790649', '0.009163127280771732']
Test Epoch0 threshold 0.4 Acc 0.9745857142857143, AUC 0.9983507394790649, avg_entr 0.009163127280771732
['0.4', '0.9745857142857143', '0.9983507394790649', '0.009163127280771732']
Test Epoch0 threshold 0.5 Acc 0.9745857142857143, AUC 0.9983507394790649, avg_entr 0.009163127280771732
['0.5', '0.9745857142857143', '0.9983507394790649', '0.009163127280771732']
Test Epoch0 threshold 0.6 Acc 0.9745857142857143, AUC 0.9983507394790649, avg_entr 0.009163127280771

In [89]:
for threshold in [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]:
    print(len(result.acc[threshold]))
    print(len(result.auc[threshold]))
    print(len(result.entr[threshold]))

10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10


In [67]:

def re_process(x):
    match = re.match('Test Epoch(.)*', x)
    return match.group()
match = df["log"][df["log"].apply(re_process)]
match

AttributeError: 'NoneType' object has no attribute 'group'

In [54]:
import re
match = re.search('Test Epoch(.)*', 'Test Epoch7 threshold 0.8 Acc 0.9749285714285715, AUC 0.9983579516410828, avg_entr 0.009062403813004494')
match.group(0)

'Test Epoch7 threshold 0.8 Acc 0.9749285714285715, AUC 0.9983579516410828, avg_entr 0.009062403813004494'

TypeError: 'float' object cannot be interpreted as an integer

In [55]:
match = re.findall("\d?\.\d+",match.group(0))
match

['0.8', '0.9749285714285715', '0.9983579516410828', '0.009062403813004494']